In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import ast

In [2]:
import pandas as pd

data_df = pd.read_csv('C:\\Users\\johnh\\OneDrive\\Documents\\공부\\2023년\\인공지능프로젝트\\기말프로젝트\\data_base.csv',
                      names=['id', 'state', 'sunlight'],
                      header=0)
print(data_df.head())
us_state = {'MN':'minnesota', 'ND':'north dakota', 'SD':'south dakota',
            'AR':'arkansas', 'CA':'california', 'LA':'louisiana',
            'IL':'illinois', 'IA':'iowa', 'NE':'nebraska'}
data_to_use = {}

   id  state  sunlight
0 NaN    NaN       NaN
1 NaN    NaN       NaN
2 NaN    NaN       NaN
3 NaN    NaN       NaN
4 NaN    NaN       NaN


In [37]:
def selective_mean(elements):
    temp_sum=0
    temp_len=0
    
    for element in elements:
        if element>0:
            temp_sum+=element
            temp_len+=1
    return temp_sum/temp_len

In [38]:
def tuple_average(*tuples):
    avg = tuple(round(selective_mean(elements),2) for elements in zip(*tuples))
    return avg

In [42]:
for city in us_state.keys():
    city_df = pd.DataFrame(index=range(88))
    with open(f"{city}.txt", 'r', encoding='utf-8') as fr:
        base_city = 'DULUTH'
        city_data = [(0,0,0,0,0,0,0,0,0,0,0,0)]*88
        while True:
            line = fr.readline()
            if not line:
                city_df[f"{base_city}"]=city_data
                #print(city_df.head(5))
                break
            line_city = line[20:33].strip()
            line_year = line[37:41]
            line_data = line[43:]
            if base_city != line_city:
                city_df[f"{base_city}"]=city_data
                #print(city_df.head(5))
                base_city = line_city
                city_data = [(0,0,0,0,0,0,0,0,0,0,0,0)]*88
            if line_year>=str(1900):
                city_data[int(line_year)-1900] = tuple([int(ele.replace('\n', '')) for ele in line_data.split(' ') if ele])
    mean_list = []
    for index in range(88):
        mean_list.append(tuple_average(*tuple(city_df.iloc[index].values)))
    city_df['mean_col']=mean_list
    city_df.to_csv(f"{city}.csv", index=False)

In [38]:
for city in us_state.keys():
    city_df = pd.read_csv(f"C:\\Users\\johnh\\OneDrive\\Documents\\공부\\2023년\\인공지능프로젝트\\기말프로젝트\\{city}.csv",
                      header=0)
    #print(city_df['mean_col'])
    years = list(range(1988, 2023))
    X = np.array(list(range(1900,1988))).reshape(-1, 1)
    
    predictions_svm = []
    predictions_reg = []
    for i in range(12):
        # 타깃 변수 생성
        y = city_df['mean_col'].apply(lambda x:ast.literal_eval(x)[i]).tolist()
        regression = LinearRegression()
        regression.fit(X, y)
        svm = SVR(kernel='rbf')
        svm.fit(X, y)
        
        # 1988년부터 2022년까지의 예측 결과 계산
        predictions_svm.append(svm.predict([[year] for year in years]))
        predictions_reg.append(regression.predict([[year] for year in years]))
    
    
    predictions_svm=np.array(predictions_svm)
    predictions_svm=np.transpose(predictions_svm).tolist()
    predictions_svm=[tuple(ele) for ele in predictions_svm]
    predictions_reg=np.array(predictions_reg)
    predictions_reg=np.transpose(predictions_reg).tolist()
    predictions_reg=[tuple(ele) for ele in predictions_reg]
    temp_df = pd.DataFrame({'SVM':predictions_svm, 'REG':predictions_reg})
    temp_df.to_csv(f"{city}_predict.csv", index=False)

In [14]:
year = list(range(1900, 2023))
month = list(range(1, 13))
ids = []

for y in year:
    for m in month:
        ids.append(f"{y}-{'{:02d}'.format(m)}")

In [10]:
from itertools import chain

In [23]:
data_df = pd.DataFrame({'id':[], 'state':[], 'sunlight_svm':[], 'sunlight_reg':[]})
data_len = 0
for state in us_state.keys():
    origin_df = pd.read_csv(f"C:\\Users\\johnh\\OneDrive\\Documents\\공부\\2023년\\인공지능프로젝트\\기말프로젝트\\{state}.csv",
                      header=0)
    predict_df = pd.read_csv(f"C:\\Users\\johnh\\OneDrive\\Documents\\공부\\2023년\\인공지능프로젝트\\기말프로젝트\\{state}_predict.csv",
                      header=0)
    origin_sunlight = list(chain.from_iterable(origin_df['mean_col'].apply(lambda x:list(ast.literal_eval(x))).tolist()))
    predict_svm_sunlight = list(chain.from_iterable(predict_df['SVM'].apply(lambda x:list(ast.literal_eval(x))).tolist()))
    #print(predict_svm_sunlight)
    predict_reg_sunlight = list(chain.from_iterable(predict_df['REG'].apply(lambda x:list(ast.literal_eval(x))).tolist()))
    
    
    origin_svm_sunlight = origin_sunlight + predict_svm_sunlight
    origin_reg_sunlight = origin_sunlight + predict_reg_sunlight
    
    state_df = pd.DataFrame({'id':ids, 'state':state, 'sunlight_svm':origin_svm_sunlight, 'sunlight_reg':origin_reg_sunlight})
    data_df = pd.concat([data_df, state_df], ignore_index=True)
    
#     for index in range(len(ids)):
#         new_row = pd.Series({'id':ids[index],'state':state, 'sunlight_svm':origin_svm_sunlight[index],'sunlight_reg':origin_reg_sunlight[index]})
#         data_df[data_len] = new_row
#         data_len+=1

data_df.to_csv(f"sunlight.csv", index=False)